In [1]:
# pip install pandas

In [2]:
# pip install requests_html

In [9]:
# pip install tensorflow

In [10]:
# pip install --upgrade pip

In [5]:
# pip install yahoo_fin

In [1]:
import ftplib
import io
import pandas as pd
import requests
import requests_html
import numpy as np
from yahoo_fin.stock_info import get_data
import tensorflow as tf

# Yahoo_fin has two modules- stock_info and options.

# stock_info has the following methods:

# get_analysts_info()
# get_balance_sheet()
# get_cash_flow()
# get_data()
# get_day_gainers()
# get_day_losers()
# get_day_most_active()
# get_holders()
# get_income_statement()
# get_live_price()
# get_quote_table()
# get_top_crypto()
# get_stats()
# get_stats_valuation()
# tickers_dow()
# tickers_nasdaq()
# tickers_other()
# tickers_sp500()

# And options has:

# get_calls()
# get_expiration_dates()
# get_options_chain()
# get_puts()

# And there are the methods you can’t use without requests_html are:

# #stock_info module
# get_day_gainers()
# get_day_most_active()
# get_day_losers()
# get_top_crypto()
# #options module
# get_expiration_dates()

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# It takes the arguments:

# ticker: case insensitive ticker of the desired stock/bond
# start_date: date you want the data to start from (mm/dd/yyyy)
# end_date: date you want the data to end (mm/dd/yyyy)
# index_as_date: {True, False}. Default is true. If true then the dates of the records are set as the index, else they are returned as a separate column.
# interval: {“1d”, “1wk”, “1mo”}. Refers to the interval to sample the data: “1d”= daily, “1wk”= weekly, “1mo”=monthly.
# get_data(ticker, start_date = None, end_date = None, index_as_date = True, interval = “1d”)

In [3]:
amazon_weekly= get_data("amzn", start_date="12/04/2009", end_date="12/04/2019", index_as_date = True, interval="1wk")
amazon_weekly

,open,high,low,close,adjclose,volume,ticker
2009-11-30,143.619995,145.910004,135.110001,137.580002,137.580002,31350900,AMZN
2009-12-07,138.000000,139.000000,129.820007,134.149994,134.149994,47863000,AMZN
2009-12-14,132.500000,132.610001,125.650002,128.479996,128.479996,45794900,AMZN
2009-12-21,130.479996,139.699997,130.190002,138.470001,138.470001,32406000,AMZN
2009-12-28,139.750000,142.580002,134.520004,134.520004,134.520004,28600700,AMZN
...,...,...,...,...,...,...,...
2019-11-04,1801.010010,1815.060059,1774.040039,1785.880005,1785.880005,11461600,AMZN
2019-11-11,1778.000000,1786.219971,1732.859985,1739.489990,1739.489990,13165500,AMZN
2019-11-18,1738.300049,1762.520020,1722.709961,1745.719971,1745.719971,13042300,AMZN
2019-11-25,1753.250000,1824.689941,1753.239990,1800.800049,1800.800049,11616400,AMZN
